## Handling Data Written in a File

This is a hands-on tutorial for reading numerical data from a file, doing a simple binning of the data, and plotting the results. The code is not optimized for efficiency, but rather for introducing some of the most useful Python data types, loops, and conditional statements. Running this notebook requires Python 3, plus the following packages:

* NumPy
* matplotlib

The ideal way to get them is to use a pre-packaged Python distribution, such as Anaconda. Anaconda comes with `NumPy` and `matplotlib` built in, as well as several of the packages required for the second part of this tutorial. You can download Anaconda from the Continuum page:
https://www.continuum.io/downloads

The code in individual cells of a Jupyter notebook (such as this one) can be executed with `Control + Enter`. If you prefer to click on virtual buttons, then you can also execute cells by clicking ![run_icon](./images/run_icon.png) in the toolbar above or by clicking on `Cell -> Run Cells`. `Control + Enter` is likely the most useful command you'll need for this tutorial, but below are a few more that could make your life easier (all for Mac keyboards):

Command | Explanation
---|---
Shift + Enter | run cell and move to cell below
⌘ + / | toggle comments for selected lines
⌘ + [ | indent selected lines
⌘ + ] | dedent selected lines
M | change cell type to markdown
Y | change cell type back to code (default)
Option + Left | move one word left
Option + Right | move one word right
Up | select cell above
Down | select cell below
Tab | code completion
Shift + Tab | view function call signature

Writing Python code generally requires sets of functions that are implemented in units called packages. Some of the packages are included in the Standard Python Library and built into Python, while others are available from the Python Package Index, GitHub, etc. Packages can be imported using the `import` command. The first three statements in the cell below load packages from the Standard Python Library. The last two load `matplotlib.pyplot` and `numpy` under the aliases `plt` and `np`, respectively. The command `%matplotlib inline` allows us to display plots in the Jupyter notebook cells.

In [ ]:
import os
import glob
from collections import OrderedDict

%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np

### Reading Data from File

We change to the directory that contains the data and set the variable `fname` to the name of the file that contains our data, in this example `glg_tte_n0_bn090510016_v00.txt`. `fname` will be a string. All strings in Python need to be in single or double quotation marks. To execute this cell without errors, you might need to modify the name of the directory where the file is located; it should be the top directory of the tutorial.

In [ ]:
os.chdir("/Users/gogrean/code/python_tutorial")
fname = "glg_tte_n0_bn090510016_v00.txt"

Below is a very general way of reading data from a file. However, there are routines in `NumPy` that can be used to do the same task in fewer lines of code. 

The files we are using here contain light curves from a GRB. First we open a file, then we handle the data line by line. Each line will be read as a string. A single line looks like this:

   -9.9979  29.7
   
where the first number is the time relative to the time of the trigger, and the second number is the energy of the photon detected at this time. We only need the first number to plot the light curve. To get it, we 

(1) split the line based on white spaces (default) to have a list of two strings--one containing the time in string format, the other the energy in string format;

(2) set `time_as_string` to the first element of the list;

(3) convert `time_as_string` to a float;

The individual `time_as_float` values are added one by one to the end of a list called `times`. The list contains the time values at which photons were detected.

In [ ]:
times = []
with open(fname) as f:
    for line in f.readlines():
        data_in_line = line.split()
        time_as_string = data_in_line[0]
        time_as_float = float(time_as_string)
        times.append(time_as_float)

Constants are usually defined in capital letters, the way we define `MIN_BIN_WIDTH` below. This constant sets the minimum width required for a time bin in the light curve.

In [ ]:
MIN_BIN_WIDTH = 0.01

One of the data structures in Python is a dictionary. Unlike lists, which are indexed by number, dictionaries are indexed by keys---like usual dictionaries. However, unlike usual dictionaries, Python dictionaries are unordered. For plotting the light curve, it does not really matter if the bins of the light curve are unordered, so we'll save the light curve in a dictionary. Each entry has the center of the bin (in units of s) as key.

In [ ]:
lc = {}
i = 0
j = 1
while j < len(times):
    if times[j] - times[i] >= MIN_BIN_WIDTH:
        c_time = 0.5 * (times[j] + times[i])
        lc[c_time] = j - i + 1
        i = j
        j += 1
    else:
        j += 1

We now change to the `data` directory and read all the light curves there. We look for files with names `glg_tte_*.txt`.

In [ ]:
os.chdir("/Users/gogrean/code/python_tutorial/data")
light_curve = []

for file in glob.glob("glg_tte_*.txt"):
    print("Working on file %s..." % file)
    times = []
    with open(file) as f:
        for line in f.readlines()[1:]:
            data_in_line = line.split()
            time_as_string = data_in_line[0]
            time_as_float = float(time_as_string)
            times.append(time_as_float)

    lc = {}
    i = 0
    j = 1
    while j < len(times):
        if times[j] - times[i] >= MIN_BIN_WIDTH:
            c_time = 0.5 * (times[j] + times[i])
            lc[c_time] = j - i + 1
            i = j
            j += 1
        else:
            j += 1
    light_curve.append(lc)

How many light curves are in the `light_curve` list?

In [ ]:
n_light_curves = len(light_curve)
print("There are %i light curves." % n_light_curves)

To keep it simple, we use only three of the light curves: `glg_tte_n6_bn090510016_v00.txt`, `glg_tte_n7_bn090510016_v00.txt`, and `glg_tte_n9_bn090510016_v00.txt`.

In [ ]:
os.chdir("/Users/gogrean/code/python_tutorial/data")
files = ["glg_tte_n6_bn090510016_v00.txt", 
         "glg_tte_n7_bn090510016_v00.txt", 
         "glg_tte_n9_bn090510016_v00.txt"]

light_curve = []

for file in files:
    print("Working on file %s..." % file)
    times = []
    with open(file) as f:
        for line in f.readlines()[1:]:
            data_in_line = line.split()
            time_as_string = data_in_line[0]
            time_as_float = float(time_as_string)
            times.append(time_as_float)

    lc = {}
    i = 0
    j = 1
    while j < len(times):
        if times[j] - times[i] >= MIN_BIN_WIDTH:
            c_time = 0.5 * (times[j] + times[i])
            lc[c_time] = (j - i + 1) / (times[j] - times[i]) / 1e4
            i = j
            j += 1
        else:
            j += 1
    light_curve.append(lc)

n_light_curves = len(light_curve)

To plot the light curves, we'll define three colors. This can be done by simply calling the colors by name, or by using the hex codes in your favorite color combo.

In [ ]:
# The easy way.
colors = ['green', 'blue', 'orange']

# The fancy way.
# Full list of available colors: 
# http://matplotlib.org/mpl_examples/color/named_colors.hires.png
colors = ['chartreuse', 'lightseagreen', 'coral']

# The super fancy way.
colors = ["#4B9E2F", "#249CFF", "#FFB52B"]

We'll do simple scatter plots of the light curves, with each of the light curves in its own subplot. Each light curve shows count rate as a function of time. This requires plotting the values (count rate) in the individual light curve dictionaries as a function of the corresponding keys (time).

In [ ]:
fig, ax = plt.subplots(n_light_curves, figsize=(12,10))
for lc, axis, color in zip(light_curve, ax, colors):
    times = list(lc.keys())
    photons = list(lc.values())
    axis.scatter(times, photons, c=color)

The plot is not looking horrible (better than default IDL plots...), but it's messy. There are a lot of overlapping data points, axis labels are missing, the font size is too small, there's a lot of unneccessary white space...

We can quickly improve the plot by defining ranges for both axis, increasing the font size, and labeling the x axis (the y axis is a little trickier to label without being super hacky, so we'll discuss later how to do it).

In [ ]:
plt.rcParams.update({'font.size': 14})

fig, ax = plt.subplots(n_light_curves, figsize=(12,10), sharex=True)
for lc, axis, color in zip(light_curve, ax, colors):
    times = list(lc.keys())
    ct_rate = list(lc.values())
    axis.scatter(times, ct_rate, marker='s', s=30, c=color)
    axis.set_xlim([0,1.3])
    axis.set_ylim([np.min(ct_rate)*0.5, np.max(ct_rate)*1.3])

plt.xlabel('Time (s)', fontsize=16)

The moment of the burst and the structure in the afterglow part of the light curve become clearer if the light curves are shown as step plots.

In [ ]:
fig, ax = plt.subplots(n_light_curves, figsize=(12,10), sharex=True)
for lc, axis, color in zip(light_curve, ax, colors):
    times = list(lc.keys())
    ct_rate = list(lc.values())
    axis.step(times, ct_rate, linewidth=3, c=color)
    axis.set_xlim([0,1.3])
    axis.set_ylim([np.min(ct_rate)*0.5, np.max(ct_rate)*1.3])

plt.xlabel('Time (s)', fontsize=16)

Let's put that tricky y label on the plot. The super hacky way to do it, since we have an odd number of plots, would be to add the label to the middle subplot so that it appears centered with all of the subplots. The more elegant way to do it is to define a big plotting area that contains the three subplots, hide the frame and the ticks, and assign the y label to its y axis. This method also ensures that if you decide to use a different number of light curves (including an even number), you will not need to manually adjust the subplot that the y label is associated with.

In [ ]:
fig, ax = plt.subplots(n_light_curves, figsize=(12,10), sharex=True, sharey=True)
for lc, axis, color in zip(light_curve, ax, colors):
    times = list(lc.keys())
    ct_rate = list(lc.values())
    axis.step(times, ct_rate, linewidth=3, c=color)
    axis.set_xlim([0,1.3])
    axis.set_ylim([np.min(ct_rate)*0.5, np.max(ct_rate)*1.3])

plt.xlabel('Time (s)', fontsize=16)

big_ax = fig.add_subplot(111, frameon=False)
big_ax.set_axis_bgcolor('none')
big_ax.tick_params(labelcolor='none', top='off', bottom='off', left='off', right='off')
big_ax.set_ylabel(r'Count rate ($10^4$ photons s$^{-1}$)', fontsize=16)

The white space between the subplots is unnecessary, so we can set it to 0.

In [ ]:
fig, ax = plt.subplots(n_light_curves, figsize=(12,10), sharex=True, sharey=True)
for lc, axis, color in zip(light_curve, ax, colors):
    times = list(lc.keys())
    ct_rate = list(lc.values())
    axis.step(times, ct_rate, c=color, linewidth=3)
    axis.set_xlim([0,1.3])
    axis.set_ylim([np.min(ct_rate)*0.5, np.max(ct_rate)*1.3])

plt.xlabel('Time (s)', fontsize=16)

big_ax = fig.add_subplot(111, frameon=False)
big_ax.set_axis_bgcolor('none')
big_ax.tick_params(labelcolor='none', top='off', bottom='off', left='off', right='off')
big_ax.set_ylabel(r'Count rate ($10^4$ photons s$^{-1}$)', fontsize=16)

fig.subplots_adjust(hspace=0)

We can also add some dashed lines to mark the position of the peaks and check that they are approximately the same in the three light curves.

In [ ]:
# Estimates for the peak times.
peak_loc = [0.54, 0.58, 0.665, 0.73, 0.825]

fig, ax = plt.subplots(n_light_curves, figsize=(12,10), sharex=True, sharey=True)
for lc, axis, color in zip(light_curve, ax, colors):
    times = list(lc.keys())
    ct_rate = list(lc.values())
    axis.step(times, ct_rate, c=color, linewidth=3)
    for peak in peak_loc:
        axis.plot([peak]*2, [0,10], c=color, linestyle='dashed', linewidth=2, alpha=0.5)
    axis.set_xlim([0,1.3])
    axis.set_ylim([np.min(ct_rate)*0.5, np.max(ct_rate)*1.3])

plt.xlabel('Time (s)', fontsize=16)

big_ax = fig.add_subplot(111, frameon=False)
big_ax.set_axis_bgcolor('none')
big_ax.tick_params(labelcolor='none', top='off', bottom='off', left='off', right='off')
big_ax.set_ylabel(r'Count rate ($10^4$ photons s$^{-1}$)', fontsize=16)

fig.subplots_adjust(hspace=0)

One important thing missing from the light curves are error bars. Adding error bars on top of a step plot looks weird, so we'll revert back to a scatter plot just to illustrate how to add simple error bars. In the last step, we'll add the uncertainty ranges on top of the step plot as a partially transparent area.

The uncertainties are not part of our current light curve dictionaries. Also, since the dictionary contains count rates rather than counts, we cannot simply take the square root of the bin values. Instead, we redefine the dictionary such that each bin still has the time as key, but has a tuple as value rather than a float; the tuple contains the count rate and the uncertainty on it.

In [ ]:
os.chdir("/Users/gogrean/code/python_tutorial/data")
files = ["glg_tte_n6_bn090510016_v00.txt", 
         "glg_tte_n7_bn090510016_v00.txt", 
         "glg_tte_n9_bn090510016_v00.txt"]
light_curve = []

for file in files:
    print("Working on file %s..." % file)
    times = []
    with open(file) as f:
        for line in f.readlines()[1:]:
            data_in_line = line.split()
            time_as_string = data_in_line[0]
            time_as_float = float(time_as_string)
            times.append(time_as_float)

    lc = {}
    i = 0
    j = 1
    while j < len(times):
        if times[j] - times[i] >= MIN_BIN_WIDTH:
            c_time = 0.5 * (times[j] + times[i])
            lc[c_time] = ((j - i + 1) / (times[j] - times[i]) / 1e4,
                          np.sqrt(j - i + 1) / (times[j] - times[i]) / 1e4)
            i = j
            j += 1
        else:
            j += 1
    light_curve.append(lc)

n_light_curves = len(light_curve)

fig, ax = plt.subplots(n_light_curves, figsize=(12,10), sharex=True, sharey=True)
for lc, axis, color in zip(light_curve, ax, colors):
    times = list(lc.keys())
    ct_rate = [x[0] for x in lc.values()]
    ct_rate_err = [x[1] for x in lc.values()]
    axis.scatter(times, ct_rate, c=color, marker='s', s=30)
    for peak in peak_loc:
        axis.plot([peak]*2, [0,10], c=color, linestyle='dashed', linewidth=2, alpha=0.5)
    axis.set_xlim([0,1.3])
    axis.set_ylim([np.min(ct_rate)*0.5, np.max(ct_rate)*1.3])
    
    axis.errorbar(times, ct_rate, yerr=ct_rate_err, color=color, fmt='s')

plt.xlabel('Time (s)', fontsize=16)

big_ax = fig.add_subplot(111, frameon=False)
big_ax.set_axis_bgcolor('none')
big_ax.tick_params(labelcolor='none', top='off', bottom='off', left='off', right='off')
big_ax.set_ylabel(r'Count rate ($10^4$ photons/s)', fontsize=16)

fig.subplots_adjust(hspace=0)

This plot looks better, but the number of data points and lines make it a bit messy. So instead we'll add the uncertainty ranges using matplotlib's `fill_between` function to 'color' between the minimum (val - sigma) and maximum (val + sigma) values of each bin. We set the transparency of this colored area to 0.5, so that we can still see the step plot.

Here, when calculating the min and the max values of each bin, the order of the bins actually matters; if the light curve bins are unordered, then the order of the elements in the lists containing the min and max values will be different than that of the elements containing the times and count rates. Therefore, we'll use an ordered dictionary, which is a data structure imported from the `collections` package. An ordered dictionary adds a new element to the end of the dictionary, so in our case the entries will be in ascending order of time.

As a last step, we'll also extract the detector name from the file name and label the subplots.

In [ ]:
os.chdir("/Users/gogrean/code/python_tutorial/data")
files = ["glg_tte_n6_bn090510016_v00.txt", 
         "glg_tte_n7_bn090510016_v00.txt", 
         "glg_tte_n9_bn090510016_v00.txt"]
light_curve = []

for file in files:
    print("Working on file %s..." % file)
    times = []
    with open(file) as f:
        for line in f.readlines()[1:]:
            data_in_line = line.split()
            time_as_string = data_in_line[0]
            time_as_float = float(time_as_string)
            times.append(time_as_float)

    lc = OrderedDict()
    i = 0
    j = 1
    while j < len(times):
        if times[j] - times[i] >= MIN_BIN_WIDTH:
            c_time = 0.5 * (times[j] + times[i])
            lc[c_time] = ((j - i + 1) / (times[j] - times[i]) / 1e4,
                          np.sqrt(j - i + 1) / (times[j] - times[i]) / 1e4)
            i = j
            j += 1
        else:
            j += 1
    light_curve.append(lc)

n_light_curves = len(light_curve)

# Get detector from file name.
detector = [s[8:10] for s in files]

fig, ax = plt.subplots(n_light_curves, figsize=(12,10), sharex=True, sharey=True)
for det, lc, axis, color in zip(detector, light_curve, ax, colors):
    times = list(lc.keys())
    ct_rate = [x[0] for x in lc.values()]
    ct_rate_err = [x[1] for x in lc.values()]
    min_ct_rate = [x[0] - x[1] for x in lc.values()]
    max_ct_rate = [x[0] + x[1] for x in lc.values()]
    axis.step(times, ct_rate, c=color, linewidth=2)
    for peak in peak_loc:
        axis.plot([peak]*2, [0,10], c=color, linestyle='dashed', linewidth=2, alpha=0.5)
    axis.set_xlim([0,1.3])
    axis.set_ylim([np.min(ct_rate)*0.5, np.max(ct_rate)*1.3])
    
    axis.fill_between(times, min_ct_rate, max_ct_rate, color=color, 
                      step='pre', alpha=0.3)
    axis.text(1.2, 0.9, det.upper(), fontsize=16)

plt.xlabel('Time (s)', fontsize=16)

big_ax = fig.add_subplot(111, frameon=False)
big_ax.set_axis_bgcolor('none')
big_ax.tick_params(labelcolor='none', top='off', bottom='off', left='off', right='off')
big_ax.set_ylabel(r'Count rate ($10^4$ photons s$^{-1}$)', fontsize=16)

fig.subplots_adjust(hspace=0)

---

*EXERCISE: Write a method to calculate the light curves from a file, and two lambda functions to calculate `min_ct_rate` and `max_ct_rate`. Re-write the code above to use these functions.*

In [ ]:
%load /Users/gogrean/code/python_tutorial/ex1_part1.py

In [ ]:
%load /Users/gogrean/code/python_tutorial/ex1_part2.py

---

*EXERCISE: Find the times corresponding to the largest jump between two consecutive bins in each of the light curves.*

In [ ]:
%load /Users/gogrean/code/python_tutorial/ex2.py

---